# 🔍 Foundry IQ: Multi-Source Fraud Investigation Agent

This notebook demonstrates **Foundry IQ** - Azure's unified knowledge layer for AI agents. You'll build a fraud investigation system that queries multiple unstructured knowledge sources through a single intelligent endpoint.

## ⚠️ Prerequisites: Verify Role Assignments

Before running this notebook, ensure your identity and the project's managed identity have the required roles on the Azure AI Search service.

### Step 0: Login to Azure

Run this command in your terminal to authenticate with Azure:

```bash
az login --use-device-code
```

### Required Roles for Azure AI Search

| Role | Assigned To | Purpose |
|------|-------------|---------|
| **Contributor** | User | General resource management (inherited from subscription) |
| **Search Index Data Contributor** | User | Create and manage indexes, upload documents |
| **Search Index Data Reader** | User + Project Managed Identity | Read access for querying indexes and knowledge base |
| **Search Service Contributor** | User | Manage search service, create knowledge bases and knowledge sources |

### Assign Roles via Azure CLI

```bash
# Replace placeholders with your values
SEARCH_SERVICE_SCOPE="/subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.Search/searchServices/<SEARCH_SERVICE_NAME>"

# For your user identity (to run this notebook)
az role assignment create --role "Search Index Data Contributor" --assignee <YOUR_USER_PRINCIPAL_ID> --scope $SEARCH_SERVICE_SCOPE
az role assignment create --role "Search Index Data Reader" --assignee <YOUR_USER_PRINCIPAL_ID> --scope $SEARCH_SERVICE_SCOPE
az role assignment create --role "Search Service Contributor" --assignee <YOUR_USER_PRINCIPAL_ID> --scope $SEARCH_SERVICE_SCOPE

# For project managed identity (required for MCP tool to access knowledge base at runtime)
az role assignment create --role "Search Index Data Reader" --assignee <PROJECT_MANAGED_IDENTITY_PRINCIPAL_ID> --scope $SEARCH_SERVICE_SCOPE
```

> **Important:** The **Search Index Data Reader** role on the **Project Managed Identity** is critical - without it, the MCP tool cannot query the knowledge base at runtime.

---

## What is Foundry IQ?

**Foundry IQ** is Microsoft's next-generation RAG (Retrieval-Augmented Generation) platform that:
- Creates **reusable knowledge bases** that ground multiple agents
- Provides **automatic source routing** - queries go to the right knowledge source(s)
- Uses **agentic retrieval** - AI plans, searches, and synthesizes across sources
- Maintains **enterprise-grade security** with document-level access control

## What You'll Build

A **Fraud Investigation Agent** with access to **3 specialized knowledge sources**:

| # | Knowledge Source | Content Type |
|---|------------------|--------------|
| 1️⃣ | **Fraud Pattern Intelligence** | Historical fraud patterns, red flags, detection strategies |
| 2️⃣ | **Regulatory Compliance** | FSI regulations, AML/KYC requirements, reporting guidelines |
| 3️⃣ | **Investigation Procedures** | Case handling protocols, evidence collection, escalation workflows |

### Key Concepts

```
┌─────────────────────────────────────────────────────────────┐
│                    FOUNDRY IQ                                │
│        (Unified Knowledge Layer for Agents)                  │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐      │
│  │ Fraud        │  │ Regulatory   │  │ Investigation│      │
│  │ Patterns     │  │ Compliance   │  │ Procedures   │      │
│  │ (Index 1)    │  │ (Index 2)    │  │ (Index 3)    │      │
│  └──────────────┘  └──────────────┘  └──────────────┘      │
│         │                 │                 │               │
│         └─────────────────┼─────────────────┘               │
│                           ▼                                  │
│              ┌─────────────────────┐                        │
│              │   Knowledge Base    │                        │
│              │ (Intelligent Router)│                        │
│              └─────────────────────┘                        │
│                           │                                  │
│                           ▼                                  │
│              ┌─────────────────────┐                        │
│              │   Fraud Analyst     │                        │
│              │      Agent          │                        │
│              └─────────────────────┘                        │
└─────────────────────────────────────────────────────────────┘
```

### References

- [Foundry IQ: Unlocking ubiquitous knowledge for agents](https://techcommunity.microsoft.com/blog/azure-ai-foundry-blog/foundry-iq-unlocking-ubiquitous-knowledge-for-agents/4470812)
- [Foundry IQ: Boost response relevance by 36% with agentic retrieval](https://techcommunity.microsoft.com/blog/azure-ai-foundry-blog/foundry-iq-boost-response-relevance-by-36-with-agentic-retrieval/4470720)
- [Create a knowledge base in Azure AI Search](https://learn.microsoft.com/en-us/azure/search/agentic-retrieval-how-to-create-knowledge-base)

---

## 📦 Step 1: Install Required Packages

The required packages should already be installed via the `requirements.txt` file at the project root:

```bash
pip install -r requirements.txt
```

If packages are not installed, uncomment and run the cell below.

In [ ]:
# Packages should be installed via requirements.txt at project root
# If not installed, uncomment the line below and run this cell:

# %pip install azure-search-documents>=11.7.0b2 azure-ai-projects>=2.0.0b1 azure-identity openai python-dotenv --quiet

print("✅ Packages ready (installed via requirements.txt)")

---

## 🔐 Step 2: Load Configuration and Connect to Azure

Load environment variables and establish connections to Azure services.

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import AzureCliCredential, get_bearer_token_provider
from azure.ai.projects import AIProjectClient
from azure.search.documents.indexes import SearchIndexClient
from openai import AzureOpenAI

# Load environment variables from parent .env
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Configuration
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model_deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
embedding_deployment = os.getenv("EMBEDDING_MODEL_DEPLOYMENT_NAME", "text-embedding-3-large")
project_resource_id = os.getenv("PROJECT_RESOURCE_ID")

# Verify required environment variables
required_vars = {
    "AI_FOUNDRY_PROJECT_ENDPOINT": project_endpoint,
    "AZURE_SEARCH_ENDPOINT": search_endpoint, 
    "AZURE_OPENAI_ENDPOINT": azure_openai_endpoint,
    "PROJECT_RESOURCE_ID": project_resource_id
}

missing = [k for k, v in required_vars.items() if not v]
if missing:
    print(f"❌ Missing environment variables: {missing}")
    print("💡 Please check your .env file")
else:
    print("✅ All required environment variables found")
    print(f"\n📍 Configuration:")
    print(f"   • Project Endpoint: {project_endpoint[:50]}...")
    print(f"   • Search Endpoint: {search_endpoint}")
    print(f"   • OpenAI Endpoint: {azure_openai_endpoint}")
    print(f"   • Model: {model_deployment}")
    print(f"   • Embedding: {embedding_deployment}")

# Create credential and clients
credential = AzureCliCredential()
print("\n🔐 Using AzureCliCredential for authentication")

# Initialize clients
index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)
openai_client = project_client.get_openai_client()

# Azure OpenAI for embeddings
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
aoai_client = AzureOpenAI(
    api_version="2024-02-01",
    azure_endpoint=azure_openai_endpoint,
    azure_ad_token_provider=token_provider
)

print("✅ All clients initialized successfully")

---

## 📊 Step 3: Generate Synthetic Fraud Investigation Data

Create realistic unstructured text data for three knowledge domains. This data is generated in-memory - no external files needed!

In [ ]:
# ============================================================================
# KNOWLEDGE SOURCE 1: Fraud Pattern Intelligence
# Unstructured narratives about fraud patterns, red flags, and detection strategies
# ============================================================================

fraud_patterns_data = [
    {
        "id": "fp-001",
        "title": "Synthetic Identity Fraud Patterns",
        "content": """Synthetic identity fraud is one of the fastest-growing types of financial crime. Fraudsters create fake identities by combining real and fabricated information - often using a legitimate Social Security Number (frequently from children, elderly, or deceased individuals) with fictitious names and addresses. Key red flags include: credit files with limited history showing sudden high-value applications, addresses that are PO boxes or commercial mail receiving agencies, phone numbers that are VoIP or recently activated, and employment information that cannot be verified. Detection strategies should focus on cross-referencing identity elements across multiple data sources, analyzing the velocity of credit applications, and using machine learning models trained on known synthetic patterns. Financial institutions report that synthetic fraud accounts often remain dormant for 12-24 months before being 'busted out' with maximum credit utilization."""
    },
    {
        "id": "fp-002", 
        "title": "Account Takeover Fraud Indicators",
        "content": """Account takeover (ATO) fraud occurs when criminals gain unauthorized access to customer accounts. Common indicators include: login attempts from new devices or geographic locations inconsistent with customer history, rapid changes to account contact information (email, phone, address) followed by high-value transactions, password reset requests from unfamiliar IP addresses, multiple failed authentication attempts before successful login, and session anomalies like unusual browsing patterns. Sophisticated ATO attacks often begin with phishing or credential stuffing. Behavioral biometrics can detect when account activity deviates from established patterns - such as typing speed, mouse movements, and navigation habits. Real-time monitoring should flag accounts where contact information changes are immediately followed by wire transfers or beneficiary additions."""
    },
    {
        "id": "fp-003",
        "title": "Money Mule Network Detection",
        "content": """Money mule networks facilitate the movement of illicit funds through seemingly legitimate bank accounts. Characteristics of mule accounts include: recently opened accounts with minimal activity followed by sudden large deposits, rapid transfers out within 24-48 hours of receiving funds, deposits that match common fraud amounts ($9,900 to avoid reporting thresholds), multiple accounts receiving funds from the same source, and account holders who are students, recently unemployed, or new to the country. Network analysis can reveal connections between mule accounts through shared attributes like device fingerprints, IP addresses, or communication patterns. Banks should implement velocity rules monitoring the frequency and amounts of incoming/outgoing transfers, particularly for accounts less than 90 days old."""
    },
    {
        "id": "fp-004",
        "title": "Business Email Compromise Patterns",
        "content": """Business Email Compromise (BEC) fraud involves impersonating executives or vendors to redirect payments. Red flags include: emails from lookalike domains with subtle misspellings, requests to change payment details for existing vendors, urgency language and secrecy requests in communications, wire transfer requests outside normal business processes, and payment routing to personal accounts or unfamiliar banks. Fraudsters often conduct reconnaissance for weeks before striking, timing attacks during executive travel or during busy periods. Prevention requires out-of-band verification for all payment changes using pre-established phone numbers, not contact information provided in the suspicious email. Financial controls should require dual authorization for payment changes and new vendor setups."""
    },
    {
        "id": "fp-005",
        "title": "First-Party Fraud Schemes",
        "content": """First-party fraud occurs when individuals misrepresent their own information for financial gain. Common schemes include: application fraud with inflated income or employment, friendly fraud where legitimate purchases are disputed as unauthorized, bust-out schemes where credit is maximized then abandoned, and insurance fraud through staged accidents or exaggerated claims. Indicators include inconsistencies between stated income and address demographics, multiple applications within short timeframes, disputes filed on high-value purchases just before return windows expire, and claims patterns inconsistent with policy history. Detection requires comparing applicant statements against third-party data sources and analyzing behavioral patterns across the customer lifecycle."""
    },
    {
        "id": "fp-006",
        "title": "Check and ACH Fraud Detection",
        "content": """Check and ACH fraud remain prevalent despite digital payment growth. Check fraud indicators include: checks deposited via mobile capture with edited images, altered payee names or amounts, counterfeit checks using real account numbers, and kiting schemes between multiple accounts. ACH fraud signs include: unauthorized debits from compromised account credentials, credits followed by rapid withdrawals before returns, and recurring debits from unfamiliar originators. Positive pay systems that match presented checks against issued check files can prevent most check fraud. ACH fraud prevention requires monitoring for first-time ACH relationships, high-value single transactions, and transactions to accounts with recent ownership changes."""
    },
    {
        "id": "fp-007",
        "title": "Elder Financial Exploitation Patterns",
        "content": """Elder financial exploitation is a growing concern in financial services. Warning signs include: sudden changes in banking patterns for long-term customers over 65, new authorized users or powers of attorney added to accounts, large withdrawals inconsistent with customer history, wire transfers to unfamiliar recipients, and signs of customer confusion during transactions. Exploitation often involves family members, caregivers, or romance scam perpetrators. Staff training should emphasize recognizing signs of undue influence, including customers appearing coached during transactions, reluctance to speak without a companion present, and fear or anxiety when discussing account activity. Suspicious activity should be reported to Adult Protective Services and may require filing SARs."""
    },
    {
        "id": "fp-008",
        "title": "Cryptocurrency-Related Fraud Schemes",
        "content": """Cryptocurrency enables new fraud vectors requiring specialized detection. Common schemes include: investment scams promising unrealistic returns, pig butchering scams building relationships before requesting crypto investments, romance scams demanding crypto payments, and fake exchange platforms that steal deposits. Red flags in traditional banking include: frequent purchases at crypto exchanges inconsistent with customer profile, rapid account funding followed by crypto purchases and depletion, customers describing 'investment advisors' met online, and requests to send wire transfers to overseas exchanges. Financial institutions should provide customer education about crypto scams and implement friction for high-risk transactions to crypto-related merchants."""
    }
]

print(f"📊 Generated {len(fraud_patterns_data)} Fraud Pattern Intelligence documents")
for doc in fraud_patterns_data[:3]:
    print(f"   • {doc['title']}")
print(f"   ... and {len(fraud_patterns_data) - 3} more")

# ============================================================================
# KNOWLEDGE SOURCE 2: Regulatory Compliance Guidelines
# FSI regulations, AML/KYC requirements, and reporting obligations
# ============================================================================

regulatory_compliance_data = [
    {
        "id": "rc-001",
        "title": "Bank Secrecy Act Requirements",
        "content": """The Bank Secrecy Act (BSA) requires financial institutions to maintain robust anti-money laundering programs. Key requirements include: establishing written AML policies and procedures approved by the board, designating a qualified BSA/AML compliance officer, implementing a customer due diligence (CDD) program, providing ongoing employee training, and conducting independent testing of the AML program at least annually. Institutions must also establish risk-based procedures for conducting ongoing customer due diligence including understanding the nature and purpose of customer relationships to develop customer risk profiles, and conducting ongoing monitoring to identify and report suspicious transactions. Failure to maintain adequate AML programs can result in civil money penalties, enforcement actions, and reputational damage."""
    },
    {
        "id": "rc-002",
        "title": "Suspicious Activity Report Filing Guidelines",
        "content": """Financial institutions must file Suspicious Activity Reports (SARs) for transactions that may involve money laundering, BSA violations, or other suspicious activities. Filing thresholds vary by institution type but generally require reporting transactions of $5,000 or more for banks and $2,000 for money services businesses when the institution knows, suspects, or has reason to suspect the transaction: involves funds from illegal activity, is designed to evade reporting requirements, lacks a lawful purpose, or involves use of the institution to facilitate criminal activity. SARs must be filed within 30 days of initial detection (60 days if no suspect is identified), must not be disclosed to subjects, and must be retained for five years. Quality SAR narratives should answer who, what, when, where, why, and how, including all relevant transaction details and supporting documentation."""
    },
    {
        "id": "rc-003",
        "title": "Customer Identification Program Standards",
        "content": """The Customer Identification Program (CIP) rule requires verification of customer identity at account opening. For individuals, institutions must collect name, date of birth, address, and identification number (SSN for US persons or passport/alien ID for non-US persons). For businesses, required information includes legal name, principal place of business address, employer identification number, and documents showing legal existence. Verification must occur within a reasonable time and may use documentary methods (government ID, business documents) or non-documentary methods (credit reports, public databases). Institutions must maintain records of identification information for five years after account closure and must check names against government lists of known terrorists. Risk-based procedures should require enhanced verification for higher-risk customers such as those in high-risk jurisdictions or industries."""
    },
    {
        "id": "rc-004",
        "title": "Enhanced Due Diligence Requirements",
        "content": """Enhanced Due Diligence (EDD) must be applied to higher-risk customers including: politically exposed persons (PEPs) and their family members, customers from high-risk jurisdictions identified by FATF, correspondent banking relationships with foreign financial institutions, private banking customers, and any relationship where standard due diligence indicates elevated risk. EDD procedures should include: senior management approval for establishing relationships, reasonable measures to establish source of wealth and source of funds, enhanced ongoing monitoring with more frequent reviews, and additional documentation requirements. For correspondent banking, EDD must assess the respondent bank's AML controls, services offered, and any nested relationships. PEP screening should cover both domestic and foreign PEPs, with ongoing monitoring for customers who become PEPs after account opening."""
    },
    {
        "id": "rc-005",
        "title": "Currency Transaction Reporting Rules",
        "content": """Financial institutions must file Currency Transaction Reports (CTRs) for cash transactions exceeding $10,000 in a single business day. This includes deposits, withdrawals, currency exchanges, and other payments made in currency. Multiple transactions must be aggregated if the institution has knowledge that transactions are conducted by or on behalf of the same person. CTRs must be filed within 15 calendar days following the transaction date. Structuring - conducting transactions to evade CTR requirements - is a federal crime even if the underlying funds are legitimate. Institutions should monitor for structuring patterns including: transactions just below $10,000, multiple branch transactions on the same day, round-dollar cash transactions, and customers who appear nervous about identification requirements during cash transactions."""
    },
    {
        "id": "rc-006",
        "title": "OFAC Sanctions Compliance",
        "content": """The Office of Foreign Assets Control (OFAC) administers economic sanctions that prohibit transactions with sanctioned countries, entities, and individuals. Financial institutions must screen customers, counterparties, and transactions against OFAC lists including the Specially Designated Nationals (SDN) list. Screening must occur at account opening, when customer information changes, when OFAC lists are updated, and for all incoming/outgoing wire transfers. Potential matches require review to confirm or rule out true matches before processing transactions. Blocked transactions must be placed in an interest-bearing account and reported to OFAC within 10 business days. Rejected transactions must also be reported. Institutions should implement a risk-based approach to sanctions compliance considering geographic exposure, customer types, and products offered. Voluntary self-disclosure of violations may result in reduced penalties."""
    },
    {
        "id": "rc-007",
        "title": "Consumer Financial Protection Requirements",
        "content": """Consumer protection regulations require fair treatment of customers in fraud investigations. Under Regulation E, consumers must report unauthorized electronic fund transfers within 60 days to limit liability, and institutions must investigate and resolve claims within 10 business days (20 days for new accounts). Provisional credit must be provided if investigations extend beyond these timeframes. For credit card fraud under Regulation Z, consumer liability is limited to $50 for unauthorized transactions reported before the card is used. Fair lending requirements prohibit discrimination in fraud investigation outcomes based on protected characteristics. Privacy regulations require protecting customer information collected during investigations. Documentation of investigation processes and outcomes must be maintained for compliance examination and potential litigation."""
    },
    {
        "id": "rc-008",
        "title": "Third-Party Risk Management for Fraud Prevention",
        "content": """Regulatory guidance requires oversight of third-party vendors used in fraud detection and prevention. Due diligence before engagement should assess the vendor's financial condition, security practices, regulatory compliance, and relevant expertise. Contracts should address data protection requirements, performance standards, audit rights, and subcontracting limitations. Ongoing monitoring should include regular performance reviews, security assessments, and verification of continued compliance. For critical vendors, institutions should maintain contingency plans addressing vendor failure or relationship termination. AI and machine learning models from vendors must be understood sufficiently to explain adverse decisions to customers and regulators. Model risk management requirements apply to third-party fraud detection models with periodic validation and documentation of model performance."""
    }
]

print(f"\n📊 Generated {len(regulatory_compliance_data)} Regulatory Compliance documents")
for doc in regulatory_compliance_data[:3]:
    print(f"   • {doc['title']}")
print(f"   ... and {len(regulatory_compliance_data) - 3} more")

# ============================================================================
# KNOWLEDGE SOURCE 3: Investigation Procedures
# Case handling protocols, evidence collection, and escalation workflows
# ============================================================================

investigation_procedures_data = [
    {
        "id": "ip-001",
        "title": "Initial Alert Triage Process",
        "content": """When a fraud alert is generated, analysts must complete initial triage within 4 hours for high-severity alerts and 24 hours for standard alerts. Triage steps include: reviewing alert details and triggering transaction(s), examining customer profile and account history, checking for related alerts or previous investigations, and documenting initial assessment in the case management system. Disposition options are: promote to investigation (with priority assignment), disposition as false positive (with documented rationale), or escalate for immediate action (for ongoing fraud or imminent loss). False positive documentation must explain why the activity is legitimate, referencing specific customer information or transaction patterns. All triage decisions require supervisor review for alerts above $25,000 or involving VIP customers."""
    },
    {
        "id": "ip-002",
        "title": "Account Freeze and Hold Procedures",
        "content": """Account freezes may be implemented when fraud is suspected to prevent further losses. Full freezes block all account activity and require compliance officer approval for accounts with balances exceeding $100,000. Partial holds restrict specific transaction types or channels while allowing essential activity. Customer notification requirements depend on freeze type and suspicion level - law enforcement holds cannot be disclosed. Documentation must include: justification for the freeze, approving authority, notification status, and planned review date. Freezes must be reviewed every 48 hours and lifted or renewed with documented justification. Excessive freeze duration without investigation progress may result in customer harm and regulatory criticism. Alternative controls like enhanced monitoring or reduced limits should be considered when full freezes cause disproportionate impact."""
    },
    {
        "id": "ip-003",
        "title": "Customer Contact Interview Techniques",
        "content": """Customer interviews are essential for distinguishing fraud from authorized activity. Before contact, review full account history, previous contacts, and alert details. Begin with open-ended questions about general account usage before addressing specific transactions. Key areas to cover: transaction purpose and counterparty relationships, device and location at time of transaction, whether credentials have been shared or compromised, and any unusual contacts or communications received. Document customer demeanor and consistency of responses. If fraud is confirmed, gather details about how compromise occurred for SAR narrative and pattern detection. If customer is suspected of first-party fraud, avoid accusatory language and focus on factual clarification. All interviews should be documented contemporaneously with quotes where relevant."""
    },
    {
        "id": "ip-004",
        "title": "Evidence Collection and Chain of Custody",
        "content": """Proper evidence handling is critical for potential law enforcement referrals and legal proceedings. Digital evidence must be preserved in original format with hash values documented. Screenshots should include timestamps, URL/system information, and analyst identification. Transaction records, communications, and system logs must be extracted through approved methods maintaining data integrity. Chain of custody documentation tracks evidence from collection through any transfers, storage, and ultimate disposition. Evidence storage must be secure with access limited to authorized personnel. Retention periods follow regulatory requirements (minimum 5 years for SAR-related materials) and legal hold requirements if litigation is anticipated. Never alter original evidence - create working copies for analysis and annotation."""
    },
    {
        "id": "ip-005",
        "title": "Loss Recovery and Chargeback Procedures",
        "content": """Loss recovery efforts should begin immediately upon confirmed fraud. For card transactions, initiate chargebacks within network timeframes (typically 120 days from transaction date). Wire recall requests must be sent within 24 hours of fraud discovery with specific information including beneficiary account, receiving bank, and fraud circumstances. ACH returns must be initiated within return timeframes (2 days for unauthorized entries). Document all recovery attempts and outcomes for reporting. Insurance claims for fraud losses require complete investigation files including loss calculations, recovery efforts, and root cause analysis. Subrogation opportunities exist when third parties contributed to the fraud. Recovery amounts should be tracked separately from gross loss figures for accurate fraud metrics."""
    },
    {
        "id": "ip-006",
        "title": "SAR Narrative Writing Standards",
        "content": """Effective SAR narratives must clearly communicate suspicious activity to FinCEN and law enforcement. Structure narratives to answer: who is conducting the suspicious activity, what instruments or mechanisms are being used, when did the activity occur (specific dates/timeframes), where did the activity take place (geographic and channel information), why is the activity suspicious (red flags and policy violations), and how was the suspicious activity committed. Include specific transaction details, amounts, dates, and counterparties. Describe the customer relationship and any relevant account history. Document investigation steps taken and findings. Avoid subjective characterizations or legal conclusions. Reference attachments containing supporting documentation. SARs for ongoing activity should be filed on a continuing basis (every 90 days) until activity ceases. Quality review by BSA staff should verify completeness and accuracy before filing."""
    },
    {
        "id": "ip-007",
        "title": "Law Enforcement Cooperation Protocols",
        "content": """Financial institutions must balance customer privacy with law enforcement cooperation. Subpoenas, summons, and search warrants require prompt response within specified timeframes - consult legal department for jurisdiction-specific requirements. Grand jury subpoenas have additional confidentiality obligations. Voluntary information sharing is permitted under the safe harbor for SAR-related information and through 314(b) registration for information sharing between financial institutions. 314(a) requests from FinCEN require searching records and reporting matches within 14 days. Direct law enforcement contacts outside formal process should be referred to designated liaison officers. Document all law enforcement contacts, requests, and responses. Testimony preparation requires legal coordination and review of investigation files. Never confirm or deny SAR filing to law enforcement or customers."""
    },
    {
        "id": "ip-008",
        "title": "Investigation Case Closure Standards",
        "content": """Case closure requires documented completion of all investigation steps and appropriate dispositions. Required elements for closure include: summary of initial alert and investigation scope, description of investigation activities and findings, conclusion regarding whether fraud occurred, quantification of actual or potential losses, documentation of SAR filing if applicable, description of recovery efforts and outcomes, recommendations for control improvements if relevant, and supervisor approval. Cases with unresolved elements require documented justification for closure with limitations noted. Closed cases must be retained according to record retention schedules (minimum 5 years for SAR-supporting documentation). Quality assurance reviews should sample closed cases for completeness and consistency. Reopen protocols exist for cases where new information emerges after closure."""
    }
]

print(f"\n📊 Generated {len(investigation_procedures_data)} Investigation Procedures documents")
for doc in investigation_procedures_data[:3]:
    print(f"   • {doc['title']}")
print(f"   ... and {len(investigation_procedures_data) - 3} more")

# Summary
total_docs = len(fraud_patterns_data) + len(regulatory_compliance_data) + len(investigation_procedures_data)
print(f"\n{'='*60}")
print(f"✅ Total: {total_docs} unstructured documents generated")
print(f"{'='*60}")

---

## 🏗️ Step 4: Create Azure AI Search Indexes

Create three vector search indexes optimized for semantic retrieval. Each index stores document embeddings (3072 dimensions) for finding semantically similar content.

In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndex, SearchField, SearchFieldDataType, VectorSearch,
    HnswAlgorithmConfiguration, VectorSearchProfile, AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration,
    SemanticPrioritizedFields, SemanticField
)
from azure.core.exceptions import ResourceNotFoundError
import time

# Define the 3 knowledge source configurations
knowledge_sources_config = {
    "fraud_patterns": {
        "index_name": "fraud-patterns-index",
        "ks_name": "fraud-patterns-ks",
        "description": "Fraud Pattern Intelligence - Historical fraud patterns, red flags, detection strategies, and behavioral indicators"
    },
    "regulatory_compliance": {
        "index_name": "regulatory-compliance-index",
        "ks_name": "regulatory-compliance-ks", 
        "description": "Regulatory Compliance - BSA/AML requirements, SAR filing guidelines, KYC procedures, OFAC sanctions"
    },
    "investigation_procedures": {
        "index_name": "investigation-procedures-index",
        "ks_name": "investigation-procedures-ks",
        "description": "Investigation Procedures - Case handling protocols, evidence collection, escalation workflows, closure standards"
    }
}

# Knowledge base name
knowledge_base_name = "fraud-investigation-kb"

# ============================================================================
# CLEANUP: Delete in correct order (KB -> KS -> Index)
# ============================================================================
print("🧹 Cleaning up existing resources (if any)...")
print("="*60)

# Step 1: Delete knowledge base first (it references knowledge sources)
try:
    index_client.get_knowledge_base(knowledge_base_name)
    print(f"🗑️ Deleting knowledge base '{knowledge_base_name}'...")
    index_client.delete_knowledge_base(knowledge_base_name)
    time.sleep(2)
    print(f"   ✅ Knowledge base deleted")
except ResourceNotFoundError:
    print(f"   ℹ️ Knowledge base '{knowledge_base_name}' not found (OK)")

# Step 2: Delete knowledge sources (they reference indexes)
for key, config in knowledge_sources_config.items():
    try:
        index_client.get_knowledge_source(config['ks_name'])
        print(f"🗑️ Deleting knowledge source '{config['ks_name']}'...")
        index_client.delete_knowledge_source(config['ks_name'])
        time.sleep(1)
        print(f"   ✅ Knowledge source deleted")
    except ResourceNotFoundError:
        print(f"   ℹ️ Knowledge source '{config['ks_name']}' not found (OK)")

print(f"\n{'='*60}")
print("✅ Cleanup complete - ready to create indexes")
print("="*60)

# ============================================================================
# CREATE INDEXES
# ============================================================================

def create_search_index(index_name):
    """Create a vector search index with hybrid search capabilities."""
    
    # Check if index exists and delete it
    try:
        index_client.get_index(index_name)
        print(f"   🗑️ Deleting existing index '{index_name}'...")
        index_client.delete_index(index_name)
        time.sleep(2)
    except ResourceNotFoundError:
        pass
    
    # Define vector search configuration
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(name="hnsw-config")
        ],
        profiles=[
            VectorSearchProfile(
                name="vector-profile",
                algorithm_configuration_name="hnsw-config",
                vectorizer_name="openai-vectorizer"
            )
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="openai-vectorizer",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=embedding_deployment,
                    model_name=embedding_deployment
                )
            )
        ]
    )
    
    # Define semantic search configuration
    semantic_search = SemanticSearch(
        default_configuration_name="semantic-config",
        configurations=[
            SemanticConfiguration(
                name="semantic-config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[SemanticField(field_name="content")]
                )
            )
        ]
    )
    
    # Define index fields
    fields = [
        SearchField(name="id", type=SearchFieldDataType.String, key=True),
        SearchField(name="title", type=SearchFieldDataType.String, searchable=True),
        SearchField(name="content", type=SearchFieldDataType.String, searchable=True),
        SearchField(
            name="content_vector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=3072,
            vector_search_profile_name="vector-profile"
        )
    ]
    
    # Create the index with vector and semantic search
    index = SearchIndex(
        name=index_name,
        fields=fields,
        vector_search=vector_search,
        semantic_search=semantic_search
    )
    
    index_client.create_or_update_index(index)

# Create all 3 indexes
print("\n🏗️ Creating Azure AI Search Indexes...")
print("="*60)

for key, config in knowledge_sources_config.items():
    print(f"\n📚 Creating: {config['index_name']}")
    create_search_index(config['index_name'])
    print(f"   ✅ Index created successfully")

print(f"\n{'='*60}")
print(f"✅ All 3 search indexes created!")

---

## 🧠 Step 5: Generate Embeddings and Upload Documents

Transform the unstructured text into vector embeddings using Azure OpenAI's text-embedding-3-large model, then upload to the search indexes.

In [ ]:
from azure.search.documents import SearchClient

def generate_embeddings(texts, batch_size=10):
    """Generate embeddings using Azure OpenAI."""
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        response = aoai_client.embeddings.create(
            input=batch,
            model=embedding_deployment
        )
        embeddings.extend([item.embedding for item in response.data])
    return embeddings

def upload_documents_to_index(index_name, documents):
    """Upload documents with embeddings to search index."""
    search_client = SearchClient(
        endpoint=search_endpoint,
        index_name=index_name,
        credential=credential
    )
    
    # Generate embeddings for all documents
    texts = [doc["content"] for doc in documents]
    embeddings = generate_embeddings(texts)
    
    # Add embeddings to documents
    for i, doc in enumerate(documents):
        doc["content_vector"] = embeddings[i]
    
    # Upload documents
    search_client.upload_documents(documents)
    return len(documents)

# Upload to all 3 indexes
print("🧠 Generating embeddings and uploading documents...")
print("="*60)

# Map data to indexes
data_mapping = {
    "fraud_patterns": fraud_patterns_data,
    "regulatory_compliance": regulatory_compliance_data,
    "investigation_procedures": investigation_procedures_data
}

for key, config in knowledge_sources_config.items():
    print(f"\n📤 Processing: {config['index_name']}")
    docs = data_mapping[key]
    
    print(f"   ⏳ Generating {len(docs)} embeddings...")
    count = upload_documents_to_index(config['index_name'], docs)
    print(f"   ✅ Uploaded {count} documents")

print(f"\n{'='*60}")
print(f"✅ All documents embedded and uploaded!")
print(f"   • Total documents: {sum(len(d) for d in data_mapping.values())}")
print(f"   • Embedding model: {embedding_deployment}")

---

## 📚 Step 6: Create Knowledge Sources

Create knowledge sources that expose each search index to the knowledge base. Knowledge sources are the abstraction layer that enables intelligent routing.

In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndexKnowledgeSource,
    SearchIndexKnowledgeSourceParameters,
    SearchIndexFieldReference
)

def create_knowledge_source(ks_name, index_name, description):
    """Create a knowledge source linked to a search index."""
    
    # Check if knowledge source exists and delete it
    try:
        index_client.get_knowledge_source(ks_name)
        print(f"   🗑️ Deleting existing knowledge source '{ks_name}'...")
        index_client.delete_knowledge_source(ks_name)
        time.sleep(1)
    except ResourceNotFoundError:
        pass
    
    # Create knowledge source
    ks = SearchIndexKnowledgeSource(
        name=ks_name,
        description=description,
        search_index_parameters=SearchIndexKnowledgeSourceParameters(
            search_index_name=index_name,
            source_data_fields=[
                SearchIndexFieldReference(name="content"),
                SearchIndexFieldReference(name="title")
            ]
        )
    )
    
    index_client.create_or_update_knowledge_source(ks)
    return ks_name

# Create all 3 knowledge sources
print("📚 Creating Knowledge Sources...")
print("="*60)

for key, config in knowledge_sources_config.items():
    print(f"\n🔗 Creating: {config['ks_name']}")
    print(f"   Description: {config['description'][:60]}...")
    
    create_knowledge_source(
        config['ks_name'],
        config['index_name'],
        config['description']
    )
    print(f"   ✅ Knowledge source created")

print(f"\n{'='*60}")
print(f"✅ All 3 knowledge sources created!")

---

## 🧠 Step 7: Create Foundry IQ Knowledge Base

Create a knowledge base that intelligently routes queries across all 3 knowledge sources. This is the power of Foundry IQ - automatic source routing based on query semantics!

In [ ]:
from azure.search.documents.indexes.models import (
    KnowledgeBase, 
    KnowledgeRetrievalMinimalReasoningEffort,
    KnowledgeRetrievalOutputMode, 
    KnowledgeSourceReference
)

# Delete existing knowledge base if present
try:
    index_client.get_knowledge_base(knowledge_base_name)
    print(f"🗑️ Deleting existing knowledge base '{knowledge_base_name}'...")
    index_client.delete_knowledge_base(knowledge_base_name)
    time.sleep(2)
except ResourceNotFoundError:
    pass

# Create references to all 3 knowledge sources
ks_references = [
    KnowledgeSourceReference(name=config['ks_name'])
    for config in knowledge_sources_config.values()
]

# Create the knowledge base with intelligent routing
# Note: retrieval_instructions requires a model - omit for EXTRACTIVE_DATA mode
knowledge_base = KnowledgeBase(
    name=knowledge_base_name,
    description="Fraud investigation knowledge base with multi-source intelligent routing",
    knowledge_sources=ks_references,
    output_mode=KnowledgeRetrievalOutputMode.EXTRACTIVE_DATA,
    retrieval_reasoning_effort=KnowledgeRetrievalMinimalReasoningEffort()
)

index_client.create_or_update_knowledge_base(knowledge_base=knowledge_base)

# Build MCP endpoint URL
mcp_endpoint = f"{search_endpoint}/knowledgebases/{knowledge_base_name}/mcp?api-version=2025-11-01-Preview"

print("="*60)
print(f"✅ Knowledge Base Created: {knowledge_base_name}")
print("="*60)
print(f"\n📊 Configuration:")
print(f"   • Name: {knowledge_base_name}")
print(f"   • Connected Sources: {len(ks_references)}")
for config in knowledge_sources_config.values():
    print(f"      - {config['ks_name']}")
print(f"   • Output Mode: EXTRACTIVE_DATA")
print(f"   • Retrieval Effort: Minimal (fast routing)")
print(f"\n📍 MCP Endpoint:")
print(f"   {mcp_endpoint}")
print(f"\n🌟 Foundry IQ Powers:")
print(f"   • Automatic query routing to correct source(s)")
print(f"   • Multi-source aggregation for complex queries")
print(f"   • Source attribution in responses")

---

## 🔐 Step 8: Create MCP Connection

Create a project connection that allows the agent to authenticate to the knowledge base MCP endpoint using managed identity.

In [ ]:
import requests

project_connection_name = "fraud-investigation-kb-connection"

bearer_token_provider = get_bearer_token_provider(credential, "https://management.azure.com/.default")
headers = {
    "Authorization": f"Bearer {bearer_token_provider()}",
    "Content-Type": "application/json"
}

# Create the MCP connection
response = requests.put(
    f"https://management.azure.com{project_resource_id}/connections/{project_connection_name}?api-version=2025-10-01-preview",
    headers=headers,
    json={
        "name": project_connection_name,
        "type": "Microsoft.MachineLearningServices/workspaces/connections",
        "properties": {
            "authType": "ProjectManagedIdentity",
            "category": "RemoteTool",
            "target": mcp_endpoint,
            "isSharedToAll": True,
            "audience": "https://search.azure.com/",
            "metadata": {"ApiType": "Azure"}
        }
    }
)

if response.status_code in [200, 201]:
    print(f"✅ MCP Connection '{project_connection_name}' created successfully")
    print(f"\n📍 Connection Details:")
    print(f"   • Name: {project_connection_name}")
    print(f"   • Auth Type: ProjectManagedIdentity")
    print(f"   • Target: Knowledge Base MCP endpoint")
else:
    print(f"⚠️ Connection creation returned: {response.status_code}")
    print(response.text)

---

## 🤖 Step 9: Create Fraud Investigation Agent

Create an AI agent with access to the Foundry IQ knowledge base via MCP tool. The agent can query all 3 knowledge sources through a single endpoint!

In [ ]:
from azure.ai.projects.models import PromptAgentDefinition, MCPTool

agent_name = "FraudInvestigationAgent"

# Agent instructions
instructions = """You are a Fraud Investigation Analyst with access to THREE specialized knowledge sources via Foundry IQ.

## YOUR KNOWLEDGE SOURCES:

🔍 **Fraud Pattern Intelligence** (fraud-patterns-ks)
- Historical fraud patterns, red flags, detection strategies
- Behavioral indicators and suspicious activity patterns

📋 **Regulatory Compliance** (regulatory-compliance-ks)
- BSA/AML requirements, SAR filing guidelines
- KYC/CDD procedures, OFAC sanctions compliance

📁 **Investigation Procedures** (investigation-procedures-ks)
- Case handling protocols, evidence collection
- Escalation workflows, closure standards

## HOW TO RESPOND:

**Step 1: ALWAYS call the knowledge_base_retrieve tool first**
- Search for relevant information before answering
- Use descriptive search queries related to the user's question

**Step 2: Read and understand the retrieved content**
- The tool will return relevant passages from the knowledge sources
- Each result includes the source name and content

**Step 3: Provide a comprehensive answer WITH CITATIONS**
- Use the retrieved information to answer the question
- Add citations in this format: 【source_name】
- Example: "Money mule accounts often show rapid transfers within 24-48 hours【fraud-patterns-ks】"

## CITATION FORMAT:
- Place citations at the end of each factual statement
- Format: 【source_name】
- Source names: fraud-patterns-ks, regulatory-compliance-ks, investigation-procedures-ks

## RESPONSE FORMAT:

For fraud detection questions:
1. Call knowledge_base_retrieve with fraud-related query
2. List the fraud indicators found with citations
3. Provide investigation recommendations with citations

For regulatory questions:
1. Call knowledge_base_retrieve with compliance query
2. Cite specific requirements with source attribution
3. Include any reporting obligations

For procedure questions:
1. Call knowledge_base_retrieve with procedure query
2. Outline step-by-step process with citations
3. Include timelines and approval requirements

## IMPORTANT:
- ALWAYS search the knowledge base first - do not answer from general knowledge
- ALWAYS include citations to show where information came from
- If the search returns relevant information, use it and cite it
- Use professional fraud investigation terminology
- Be helpful and provide actionable guidance"""

# Create MCP tool connecting to knowledge base
mcp_tool = MCPTool(
    server_label="fraud-kb",
    server_url=mcp_endpoint,
    require_approval="never",
    allowed_tools=["knowledge_base_retrieve"],
    project_connection_id=project_connection_name
)

# Create the agent
agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment,
        instructions=instructions,
        tools=[mcp_tool]
    )
)

# Store the version for reference
agent_version = agent.version

print("="*60)
print(f"✅ Fraud Investigation Agent Created")
print("="*60)
print(f"\n📊 Agent Configuration:")
print(f"   • Name: {agent.name}")
print(f"   • ID: {agent.id}")
print(f"   • Version: {agent.version}")
print(f"   • Model: {model_deployment}")
print(f"   • Knowledge Base: {knowledge_base_name}")
print(f"   • MCP Tool: knowledge_base_retrieve")
print(f"\n🔖 Use version {agent.version} in the Microsoft Foundry UI")

---

## 🧪 Step 10: Test the Fraud Investigation Agent

Test the agent with queries that span different knowledge sources. Foundry IQ will automatically route each query to the appropriate source(s)!

In [ ]:
# Verify agent version being used
print(f"🤖 Using Agent: {agent.name} (Version: {agent.version})")
print(f"   Agent ID: {agent.id}")

# Create a conversation
conversation = openai_client.conversations.create()
print(f"✅ Conversation created: {conversation.id}\n")

# Test questions demonstrating multi-source routing
test_queries = [
    {
        "question": "What are the key red flags for synthetic identity fraud?",
        "expected_source": "Fraud Patterns",
        "emoji": "🔍"
    },
    {
        "question": "When must I file a Suspicious Activity Report and what should it contain?",
        "expected_source": "Regulatory Compliance",
        "emoji": "📋"
    },
    {
        "question": "What's the proper procedure for freezing a suspicious account?",
        "expected_source": "Investigation Procedures", 
        "emoji": "📁"
    },
    {
        "question": "I found a potential money mule network - what are the patterns, regulations, and investigation steps?",
        "expected_source": "ALL THREE SOURCES",
        "emoji": "🌟"
    }
]

print("="*70)
print("🧪 TESTING FOUNDRY IQ INTELLIGENT ROUTING")
print("="*70)

for i, test in enumerate(test_queries, 1):
    print(f"\n{'─'*70}")
    print(f"{test['emoji']} TEST {i}: Query routing to {test['expected_source']}")
    print(f"{'─'*70}")
    print(f"\n💬 Question: {test['question']}")
    print(f"\n⏳ Agent searching knowledge base...")
    
    # Use agent name AND version explicitly
    response = openai_client.responses.create(
        conversation=conversation.id,
        tool_choice="required",
        input=test['question'],
        extra_body={
            "agent": {
                "name": agent.name, 
                "version": agent.version,
                "type": "agent_reference"
            }
        },
    )
    
    print(f"\n🤖 Response:")
    print(f"{'─'*70}")
    # Truncate long responses for readability
    output = response.output_text
    if len(output) > 1500:
        output = output[:1500] + "\n\n... [truncated for display]"
    print(output)
    
    time.sleep(2)  # Add delay between requests

print(f"\n{'='*70}")
print("✅ All tests complete!")
print("="*70)
print("\n🌟 Key Observations:")
print("   • Foundry IQ automatically routes queries to correct knowledge source(s)")
print("   • Complex queries span multiple sources seamlessly")
print("   • No manual source selection needed - it's all automatic!")
print("   • Source attribution shows where information came from")

---

## 🧹 Step 11: Cleanup Resources

Delete all created resources. Uncomment the code in the next cell to run cleanup.

In [ ]:
# ============================================================================
# CLEANUP - Uncomment to delete all resources
# ============================================================================

# Delete the agent
# project_client.agents.delete_version(agent.name, agent.version)
# print(f"✅ Deleted agent: {agent.name}")

# Delete the knowledge base
# index_client.delete_knowledge_base(knowledge_base_name)
# print(f"✅ Deleted knowledge base: {knowledge_base_name}")

# Delete knowledge sources
# for config in knowledge_sources_config.values():
#     try:
#         index_client.delete_knowledge_source(config['ks_name'])
#         print(f"✅ Deleted knowledge source: {config['ks_name']}")
#     except: pass

# Delete indexes
# for config in knowledge_sources_config.values():
#     try:
#         index_client.delete_index(config['index_name'])
#         print(f"✅ Deleted index: {config['index_name']}")
#     except: pass

print("💡 Uncomment the code above to cleanup resources")

---

## 📚 Summary & References

### What You've Built

You created a **Foundry IQ-powered Fraud Investigation Agent** with:

| Component | Description |
|-----------|-------------|
| **3 Search Indexes** | Vector-enabled indexes for semantic search |
| **3 Knowledge Sources** | Fraud patterns, regulatory compliance, investigation procedures |
| **1 Knowledge Base** | Intelligent routing layer connecting all sources |
| **1 AI Agent** | Fraud analyst with MCP tool access to knowledge base |

### Key Foundry IQ Capabilities Demonstrated

1. **Automatic Source Routing** - Queries automatically go to the right knowledge source(s)
2. **Multi-Source Aggregation** - Complex queries span multiple sources seamlessly  
3. **Agentic Retrieval** - AI plans, searches, and synthesizes across sources
4. **Source Attribution** - Responses include citations showing information origin

### References

- [Foundry IQ: Unlocking ubiquitous knowledge for agents](https://techcommunity.microsoft.com/blog/azure-ai-foundry-blog/foundry-iq-unlocking-ubiquitous-knowledge-for-agents/4470812)
- [Foundry IQ: Boost response relevance by 36% with agentic retrieval](https://techcommunity.microsoft.com/blog/azure-ai-foundry-blog/foundry-iq-boost-response-relevance-by-36-with-agentic-retrieval/4470720)
- [Create a knowledge base in Azure AI Search](https://learn.microsoft.com/en-us/azure/search/agentic-retrieval-how-to-create-knowledge-base)
- [Azure AI Search Documentation](https://learn.microsoft.com/azure/search/)

---

**🎉 Congratulations!** You've successfully built a multi-source knowledge agent using Foundry IQ!

*Note: This is for educational purposes. Production deployments require proper error handling, security, and compliance features.*